In [ ]:
%pip install httpx pandas pandas_ta matplotlib seaborn plotly requests beautifulsoup4 lxml mplfinance finnhub-python binance python-binance scikit-learn numpy rich marketwatch

In [ ]:
import time
import pandas as pd
from marketwatch import MarketWatch
import logging

In [ ]:
# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(df, periods):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=periods).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(0)  # Handling NaN values

In [ ]:
# Function to execute trading strategy based on RSI
def trade_on_rsi_strategy(df, marketwatch, game_name, stock_symbol):
    rsi = df['RSI_14'].iloc[-1]

    # Buy condition - RSI below 30 indicates potential oversold conditions
    if rsi < 30:
        logging.info(f'RSI below 30 for {stock_symbol}. Attempting to buy stock.')
        marketwatch.buy(game_name, stock_symbol, 100)

        # Covering short position if previously shorted
        marketwatch.cover_short(game_name, stock_symbol, 100)
        logging.info(f'Covered short position for {stock_symbol}.')
    
    # Sell condition - RSI above 70 indicates potential overbought conditions
    elif rsi > 70:
        logging.info(f'RSI above 70 for {stock_symbol}. Attempting to sell stock.')
        marketwatch.sell(game_name, stock_symbol, 100)

        # Initiating a short sell
        marketwatch.short_sell(game_name, stock_symbol, 100)
        logging.info(f'Initiated a short sell for {stock_symbol}.')

In [ ]:
# Function to collect and process stock price data
def trade_multiple_stocks(stock_symbols, marketwatch, game_name):
    price_data = {symbol: [] for symbol in stock_symbols}

    for _ in range(160):
        for symbol in stock_symbols:
            try:
                price_pattern = re.compile(r'\b' + re.escape(symbol) + r' : \$([\d\.]+)\b')
                price_string = marketwatch.get_price(symbol)
                price_match = price_pattern.search(price_string)
                if price_match:
                    price = float(price_match.group(1))
                    price_data[symbol].append(price)
                else:
                    print(f"Received price string: {price_string}")
                    raise ValueError(f"Unexpected price format for {symbol}: {price_string}")
            except Exception as e:
                print(f"Unable to parse price for symbol: {symbol}, received: {price_string}, error: {e}")
                continue

            if len(price_data[symbol]) > 150:
                price_data[symbol].pop(0)

            df = pd.DataFrame(price_data[symbol], columns=['Close'])
            df['RSI_14'] = calculate_rsi(df, 14)

            if len(df) >= 14:  # Ensure we have enough data for RSI calculation
                trade_on_rsi_strategy(df, marketwatch, game_name, symbol)

        time.sleep(60)  # Adjust this as per your requirement

In [ ]:
def next_trading_day():
    today = datetime.today()
    if today.weekday() >= 5:  # Saturday or Sunday
        return today + timedelta(days=(7 - today.weekday()))
    return today

In [ ]:
def main():
    username = os.getenv('MARKETWATCH_USERNAME')
    password = os.getenv('MARKETWATCH_PASSWORD')
    if not username or not password:
        logging.error("Username or password not set in environment variables.")
        return

    game_name = 'marketwatchapistrategieema'
    marketwatch = MarketWatch(username, password)

    next_trade_day = next_trading_day()
    if datetime.today() < next_trade_day:
        sleep_seconds = (next_trade_day - datetime.today()).total_seconds()
        logging.info(f"Today is a non-trading day. Sleeping until {next_trade_day}")
        time.sleep(sleep_seconds)

    tech_stocks = ['AAPL', 'GOOGL']
    health_stocks = ['JNJ']
    utility_stocks = ['NEE']
    all_stocks = tech_stocks + health_stocks + utility_stocks

    trade_multiple_stocks(all_stocks, marketwatch, game_name)

if __name__ == "__main__":
    main()